In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT

densenet_path = '../input/densenet_test.h5'
BATCHES_COUNT = 12091
TEST_SIZE = 1768182

In [4]:
%%time
store = pd.HDFStore(densenet_path)
densenet_keys = store.keys()
print('{} tables in storage'.format(len(densenet_keys)))

0 tables in storage
CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 4.6 ms


# Усреднить только денснет

In [ ]:
assert len(keys) == BATCHES_COUNT

dense_preds = []
cur_id = None
cur_sum = None

for key in tqdm(densenet_keys): 
    table = store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    assert table.shape[0] % 10 == 0
    for i in range(int(table.shape[0] / 10)):
        product_id = table.pr_id.iloc[10*i]
        new_sum = table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        
        if cur_id == product_id:
            assert cur_sum is not None
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            dense_preds.append((int(cur_id), cat))
            
        cur_id = product_id
        cur_sum = new_sum
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    dense_preds.append((int(cur_id), cat))
            
dense_preds_df = pd.DataFrame(data=dense_preds, columns=['_id','category_id'])
print('Dataframe shape {}; products in test {}'.format(dense_preds_df.shape, TEST_SIZE))
print('Duplicates {}; affect accuracy {}'.format(dense_preds_df.shape[0] - TEST_SIZE,
                                                float(dense_preds_df.shape[0] - TEST_SIZE) / TEST_SIZE))
dense_no_dupls = dense_preds_df.drop_duplicates(subset='_id', inplace=False)
print('No duplicated shape:', dense_no_dupls.shape)

dense_no_dupls.to_csv('../submit/dense_all_imgs.csv', index=False)

# Усреднить денснет с резнетом

In [5]:
%%time
res_store = pd.HDFStore('../input/predict_probs_resnet50.h5')
res_keys = res_store.keys()
print('{} tables in storage'.format(len(res_keys)))

0 tables in storage
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.55 ms


In [ ]:
ave_preds = []
cur_id = None
cur_sum = None

for key in tqdm(densenet_keys): 
    res_table = res_store.select(key)
    dens_table = store.select(key)
    # table.drop(table[table.img_num != 0.].index, axis=0, inplace=True)
    assert res_table.shape[0] % 10 == 0
    assert dens_table.shape[0] == res_table.shape[0]
    for i in range(int(res_table.shape[0] / 10)):
        product_id1 = res_table.pr_id.iloc[10*i]
        product_id2 = dens_table.pr_id.iloc[10 * i]
        assert product_id1 == product_id2
        new_sum1 = res_table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        new_sum2 = dens_table[10*i : 10*i + 10].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        new_sum = new_sum1 + new_sum2
        
        if cur_id == product_id1:
            assert cur_sum is not None
            cur_sum += new_sum
            continue
        elif cur_id is not None:
            assert cur_sum is not None
            id_ = int(cur_sum.idxmax()) - 1
            cat = LABEL_TO_CAT[id_]
            ave_preds.append((int(cur_id), cat))
            
        cur_id = product_id1
        cur_sum = new_sum
        
if cur_id is not None:
    id_ = int(cur_sum.idxmax()) - 1
    cat = LABEL_TO_CAT[id_]
    ave_preds.append((int(cur_id), cat))
            
ave_preds_df = pd.DataFrame(data=ave_preds, columns=['_id','category_id'])
print('Dataframe shape {}; products in test {}'.format(ave_preds_df.shape, TEST_SIZE))
print('Duplicates {}; affect accuracy {}'.format(ave_preds_df.shape[0] - TEST_SIZE,
                                                float(ave_preds_df.shape[0] - TEST_SIZE) / TEST_SIZE))
ave_no_dupls = ave_preds_df.drop_duplicates(subset='_id', inplace=False)
print('No duplicated shape:', ave_no_dupls.shape)

ave_no_dupls.to_csv('../submit/dense_resnet_ave.csv', index=False)